In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import warnings
import torch
import optuna

ct_utils = importlib.import_module('ct_utils');

warnings.filterwarnings("ignore");
print('Pandas version:', pd.__version__);
print('Numpy version:', np.__version__);
print('MatplotLib version:', mpl.__version__);
print('Sklearn version:', sklearn.__version__);
print('Seaborn version:', sns.__version__);
print('PyTorch version:', torch.__version__);
print('Optuna version:', optuna.__version__);

# Reading the datasets for train and test


### The following lists need to be specified:

In [ ]:
melloddy_emb_cols = ["melloddy_emb_" + str(i) for i in range(2000)]; #columns specifying the structural encoding

#additional input columns (e.g. the species) have to be added to the global features list
global_features = melloddy_emb_cols;

time_cols = [ "time_" + str(i) for i in range(100)]; #timestamps of the measurments
conc_cols_po = [ "conc_po_" + str(i) for i in range(100)]; # po concentration measurments in nM 
conc_cols_iv = [ "conc_iv_" + str(i) for i in range(100)]; # iv concentration measurments in nM 

target_features =  time_cols + conc_cols_po + conc_cols_iv;

dose_column = "Dose" # mg/kg
mass_column = "Average Mass" # g/mol
species_colum = "Species" # Rat

input_data = "dummy.csv"
output_folder = "tmp_folder"

In [ ]:
#generate dummy data
df = pd.DataFrame();
df[melloddy_emb_cols+time_cols+conc_cols_po+conc_cols_iv + ["Dose","Average Mass"]] = np.random.rand(1000,2000+300+2);
df[species_colum] = ["Rat"]*df.shape[0];
df.to_csv("dummy.csv");

In [ ]:
# load the data
# each row is one experiment, i.e. one PK study
df = pd.read_csv(input_data);

In [ ]:
df["Dose_trf"] = ct_utils.recalc_dose(df[dose_column].to_numpy(dtype=float), df[mass_column].to_numpy(), df[species_colum].to_numpy());
df = df.dropna(subset = time_cols + conc_cols_po + conc_cols_iv, how="all");

In [ ]:
df_test = df.iloc[int(0.8*df.shape[0]):];
df_train = df.iloc[:int(0.8*df.shape[0])];
df_val = df_test.iloc[int(0.75*df_test.shape[0]):];
df_test = df_test.iloc[:int(0.75*df_test.shape[0])];

## Scale (not necessary when only MELLODDY embeddings used as input)

In [ ]:
"""from sklearn.preprocessing import StandardScaler

features_to_be_normed = ["Dose_trf"]

sc = StandardScaler()
final_train_scaler = sc.fit(df_train[features_to_be_normed]);
import pickle
scalerfile = output_folder + '/scaler.sav'
pickle.dump(final_train_scaler, open(scalerfile, 'wb'))
final_train_scaler = pickle.load(open(scalerfile, 'rb'))

df_train[features_to_be_normed] = final_train_scaler.transform(df_train[features_to_be_normed]);
df_val[features_to_be_normed] = final_train_scaler.transform(df_val[features_to_be_normed]);
df_test[features_to_be_normed] = final_train_scaler.transform(df_test[features_to_be_normed]);"""

# Define the MLP

In [ ]:
torch.autograd.set_detect_anomaly(True)
from torch.utils.data import DataLoader
from torch.autograd import Variable
from sklearn import metrics
 
if torch.cuda.is_available():
    print('use GPU');
    device='cuda';
else:
    print('use CPU');
    device='cpu';
    
mod = importlib.import_module('DeepCt');
importlib.reload(mod);
DeepCt = getattr(mod, 'DeepCt');

ct_utils = importlib.import_module('ct_utils');
importlib.reload(ct_utils);

In [ ]:
def get_data( global_features, targets):

    global_feat_list = [];
    for i in range(targets.shape[0]):
                global_feat_list.append(torch.from_numpy(global_features[i,:])); 
        
    return  global_feat_list, targets;

In [ ]:
def collate(sample):
    global_feats, labels = map(list,zip(*sample));
    global_feats = torch.stack([torch.tensor(tmp) for tmp in global_feats]);

    return  global_feats, torch.tensor(labels);

In [ ]:
def fit_final(params, verbose=True):
    import copy
    print(params);
    model = DeepCt( global_feats=len(global_features), predictor_dropout=params["dropout"], num_layers=params["depth"],
                                       n_tasks=len(ct_utils.targets_combined), predictor_hidden_feats=params["predictor_hidden_feats"]);

    model = model.to(device);

    global_feat_train, targets_train = get_data( df_train[global_features].to_numpy(dtype=np.float32), 
                                                                 df_train[target_features + ["Dose_trf"] ].to_numpy());
    
    global_feat_val, targets_val = get_data( df_val[global_features].to_numpy(dtype=np.float32),
                                                           df_val[target_features + ["Dose_trf"] ].to_numpy());
    if verbose:
        print("Data loaded ...");
    
    train_data = list(zip( global_feat_train, targets_train)).copy();
    train_loader = DataLoader(train_data, batch_size=int(params["batch_size"]), shuffle=True, collate_fn=collate, pin_memory=True, num_workers=1, drop_last=True);
    
    val_data = list(zip( global_feat_val, targets_val)).copy();
    val_loader = DataLoader(val_data, batch_size=256, shuffle=False, collate_fn=collate, drop_last=False);

    optimizer = torch.optim.AdamW(model.parameters(), weight_decay=params["weight_decay"]);
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.001, mode="triangular2", step_size_up=2*len(train_loader),cycle_momentum=False);
    model.train();

    epoch_losses = [];
    epoch_accuracies = [];
    
    #early stopping params
    num_no_improvements = 0;
    best_val = 10e20;
    patience = 10;
        
    for epoch in range(1,100):
        model.train();
        epoch_loss = 0;
        for i, (global_feats, labels) in enumerate(train_loader):
            
            labels = labels.to(device);
            global_feats = global_feats.to(device);
            
            preds = model( global_feats);
            
            loss = ct_utils.L2_expcurve_and_readout_loss(preds, labels.float(), readout_weight=0.0, num_cmpts=2);

            optimizer.zero_grad();
            loss.backward();
                        
            optimizer.step();
            scheduler.step();
            
        #######################
        ### Valid the model ###
        #######################
        model.eval();
        pred_es = np.empty(shape=[0, len(ct_utils.targets_combined)]);
        for i, ( global_feats, labels) in enumerate(val_loader):
            global_feats = global_feats.to(device);
            labels = labels.to(device);
            tmp_preds = model( global_feats);

            tmp_preds = tmp_preds.cpu().detach().numpy();    
            pred_es = np.append(pred_es, tmp_preds, axis=0);
            
        tmp_val = ct_utils.L2_expcurve_and_readout_loss(torch.from_numpy(pred_es), 
                                                        torch.from_numpy(df_val[target_features + ["Dose_trf"]].to_numpy()), 
                                                        readout_weight=0.0, num_cmpts=2);
        
        tmp_val = tmp_val.detach().numpy();
        if verbose:
            print("Learning rate: " + str(optimizer.param_groups[0]['lr']));

        if tmp_val < best_val:
            num_no_improvements = 0;
            best_val = tmp_val;
            best_model = copy.deepcopy(model);
        else:
            num_no_improvements = num_no_improvements + 1;
            if num_no_improvements>patience:
                if verbose:
                    print("Early stopping here ...");
                break;
        
        epoch_accuracies.append(tmp_val);
        if verbose:
            print("Current Test-Loss: " + str(tmp_val));
      
    if verbose:
        print("Best val: " + repr(best_val));
    
    return best_model, best_val;

# Run the training

### Hyperparameter optimization with Optuna

In [ ]:
num_trials = 3;
study_name = "dummy";
def objective(trial):
    
    #set the current hyperparameters
    final_params = {'batch_size': 2**(trial.suggest_int("batch_size", 4, 10, step=1)), 'depth': trial.suggest_int("depth", 3, 10), 'dropout': trial.suggest_float("dropout", 0.0, 0.9, step=0.05), 
                    'predictor_hidden_feats': 2**(trial.suggest_int("predictor_hidden_feats", 4, 10, step=1)), 'weight_decay': trial.suggest_float("weight_decay", 0.0, 0.1, step=0.01)};
    
    #train the model
    model, loss = fit_final(final_params, verbose=False);
    
    return loss

study = optuna.create_study(study_name=study_name);
study.optimize(objective, n_trials=num_trials);
    
study.trials_dataframe().to_csv("optuna_" + study_name + ".csv");
print('Best value: {} (params: {})\n'.format(study.best_value, study.best_params));

***IMPORTANT***: batch_size as well as predictor_hidden_feats are optimized with respect to base 2. E.g. the actual batch size is 2^y when y is the result from optuna

### Final training

In [ ]:
import copy

final_params = {'batch_size': 32, 'depth': 5, 'dropout': 0.3, 'predictor_hidden_feats': 256, 'weight_decay': 0.01};

best_loss = 10e20;
i = 0;

while i < 10:
    model, tmp_loss = fit_final(final_params);
    torch.save(model.state_dict(), output_folder +  "/weights_" + str(i) + ".pth");
    i = i + 1;
    if tmp_loss < best_loss:
        best_loss = tmp_loss;
        best_model = copy.deepcopy(model);